In [21]:
import networkx as nx
import networkx.algorithms.approximation as approx
from networkx.algorithms.tree.mst import random_spanning_tree
import tsplib95

import pulp

import itertools

problem = tsplib95.load("br17.atsp")

G = problem.get_graph()

G.is_directed() 

True

In [22]:

# only keep weights: (i,j,{'weight': weight, 'is_fixed": True}) -> G[i][j] = weight
for u, v, data in G.edges(data=True):
    weight = data.get('weight', 0)  
    G[u][v].clear()  
    G[u][v]['weight'] = weight

# making sure G is complete

for i in G.nodes:
    for j in G.nodes:
        if i != j and not G.has_edge(i, j):
            G.add_edge(i, j, weight=9999)
            print(f"[!] added edge {i} {j} with weight 9999")

G.edges(data=True)



OutEdgeDataView([(0, 0, {'weight': 9999}), (0, 1, {'weight': 3}), (0, 2, {'weight': 5}), (0, 3, {'weight': 48}), (0, 4, {'weight': 48}), (0, 5, {'weight': 8}), (0, 6, {'weight': 8}), (0, 7, {'weight': 5}), (0, 8, {'weight': 5}), (0, 9, {'weight': 3}), (0, 10, {'weight': 3}), (0, 11, {'weight': 0}), (0, 12, {'weight': 3}), (0, 13, {'weight': 5}), (0, 14, {'weight': 8}), (0, 15, {'weight': 8}), (0, 16, {'weight': 5}), (1, 0, {'weight': 3}), (1, 1, {'weight': 9999}), (1, 2, {'weight': 3}), (1, 3, {'weight': 48}), (1, 4, {'weight': 48}), (1, 5, {'weight': 8}), (1, 6, {'weight': 8}), (1, 7, {'weight': 5}), (1, 8, {'weight': 5}), (1, 9, {'weight': 0}), (1, 10, {'weight': 0}), (1, 11, {'weight': 3}), (1, 12, {'weight': 0}), (1, 13, {'weight': 3}), (1, 14, {'weight': 8}), (1, 15, {'weight': 8}), (1, 16, {'weight': 5}), (2, 0, {'weight': 5}), (2, 1, {'weight': 3}), (2, 2, {'weight': 9999}), (2, 3, {'weight': 72}), (2, 4, {'weight': 72}), (2, 5, {'weight': 48}), (2, 6, {'weight': 48}), (2, 7, {'

In [23]:
def HK(G):
    """
    Compute the Held-Karp relaxation for the ATSP following Asadpour's formulation.
    """
    if not G.is_directed():
        raise ValueError("Graph must be a directed graph (DiGraph).")

    V = list(G.nodes)
    A = list(G.edges)
    c = nx.get_edge_attributes(G, 'weight')

    prob = pulp.LpProblem("Held-Karp_Relaxation", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", A, lowBound=0, cat='Continuous')

    # objective
    prob += pulp.lpSum([c[arc] * x[arc] for arc in A]), "Total_Cost"

    # flow conservation constraints (3.3)
    for v in V:
        prob += pulp.lpSum([x[(v, w)] for w in G.successors(v)]) == 1, f"Outflow_{v}"
        prob += pulp.lpSum([x[(u, v)] for u in G.predecessors(v)]) == 1, f"Inflow_{v}"

    # subset cut constraints (3.2)
    for size in range(1, len(V)):
        for U in itertools.combinations(V, size):
            U = set(U)
            if len(U) < len(V):  
                delta_plus_U = [(u, v) for u in U for v in G.successors(u) if v not in U]
                prob += pulp.lpSum([x[arc] for arc in delta_plus_U]) >= 1, f"Cut_{U}"


    prob.solve()

    # extract x_star
    x_star = {arc: (x[arc].varValue or 0) for arc in A}

    # post-processing and scaling to z_star (3.5)
    z_star = {}
    scale_factor = (len(V) - 1) / len(V)  # (n-1)/n scaling factor from paper
    for (u, v) in A:
        frequency = x_star[(u, v)] + x_star.get((v, u), 0)
        if frequency > 0:
            z_star[(u, v)] = scale_factor * frequency

    # just some sanity checks
    support_graph = nx.Graph()
    for (u, v), val in z_star.items():
        if val > 0:
            support_graph.add_edge(u, v)
    
    if not nx.is_connected(support_graph):
        raise ValueError("Held-Karp solution produced disconnected support graph!")

    return pulp.value(prob.objective), z_star

In [24]:
HK_opt, z_star = HK(G)

print("Held-Karp relaxation optimal cost:", HK_opt)
print("Symmetrized solution:", z_star)

if isinstance(z_star, nx.DiGraph):
    print(f"z_star is digraph: {z_star}")
    for u,v in z_star.edges:
        print(u,v, G[u][v]['weight'])

else:
    print(f"z_star is dict: {z_star}")
    for u,v in z_star.keys():
        if u < v: 
            print(u, v, z_star[(u, v)])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/chuck/miniforge3/envs/CS632/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tf/kyhcn0rj15722n_6n2h54bcc0000gn/T/731b2d8ed90343e899d830bacf782955-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/tf/kyhcn0rj15722n_6n2h54bcc0000gn/T/731b2d8ed90343e899d830bacf782955-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 131109 COLUMNS
At line 9044837 RHS
At line 9175942 BOUNDS
At line 9175943 ENDATA
Problem MODEL has 131104 rows, 289 columns and 8913474 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
would have 0 free columns in primal, 34 in dual
Dual of model has 289 rows and 131104 columns
Coin0506I Presolve 289 (0) rows, 131104 (0) columns and 8913474 (0) elements
Clp0014I Perturbing problem by 0.001% of 1 - largest nonzero change 9.9997604e-05 ( 0.0099997604%) - larg

In [25]:
if not isinstance(z_star, dict):
    print(_shortcutting(nx.eulerian_circuit(z_star)))
    exit()

In [26]:
from math import exp 
import numpy as np

def get_q(G, gamma):
    node_list = list(G.nodes())
    node_index = {node: i for i, node in enumerate(node_list)}
    
    for u, v, d in G.edges(data=True):
        d["lambda"] = exp(gamma[(u, v)])
    
    # Construct weighted Laplacian using lambda
    L = nx.laplacian_matrix(G, weight='lambda').toarray()

    # Compute pseudoinverse of the Laplacian
    L_pinv = np.linalg.pinv(L)
    
    # Compute q_e for each edge and store in a dictionary
    q = {}
    for u, v, d in G.edges(data=True):
        ui, vi = node_index[u], node_index[v]
        R_uv = L_pinv[ui, ui] + L_pinv[vi, vi] - 2 * L_pinv[ui, vi]
        q[(u, v)] = d['lambda'] * R_uv
    
    return q

In [27]:
from math import log as ln

def spanning_tree_distribution(G, z):
    # Initialize gamma to zero for each arc
    gamma = {arc: 0 for arc in G.edges()}

    # Set epsilon
    EPSILON = 0.2

    while True:
        # Search for an edge with q_e > (1 + epsilon) * z_e
        for e in gamma.keys():
            q_e = get_q(G, gamma)[e]
            z_e = z[e]
            if q_e > (1 + EPSILON) * z_e:
                # Lemma 7.1
                delta = ln(
                    (q_e * (1 - (1 + EPSILON / 2) * z_e))
                    / ((1 - q_e) * (1 + EPSILON / 2) * z_e)
                )
                # Check that delta had the desired effect on q
                # we decrease gamma_e in order to decrease qe(gamma) to (1 +epsilon/2) * ze
                gamma[e] -= delta
                new_q_e = get_q(G, gamma)[e]
                desired_q_e = (1 + EPSILON / 2) * z_e
                if round(new_q_e, 8) != round(desired_q_e, 8):
                    raise nx.NetworkXError(
                        f"Unable to modify probability for edge ({u}, {v})"
                    )
        # Check terminate condition
        q = get_q(G, gamma)
        if all(q[e] <= (1 + EPSILON) * z[e] for e in G.edges()):
            break

    # Remove lambda from edge attribute
    for _, _, d in G.edges(data=True):
        if "lambda" in d:
            del d["lambda"]

    return gamma

In [28]:
z_support = nx.MultiGraph()
for u, v in z_star:
    if (u, v) not in z_support.edges:
        edge_weight = min(G[u][v]['weight'], G[v][u]['weight'])
        z_support.add_edge(u, v, weight=edge_weight)


print(f"==================== printing z_support =====================")
for u,v in z_support.edges():
    print(min(u,v), max(u,v), z_support[u][v][0]['weight'])

print(f"==================== printing z_star (dict) =====================")
for u,v in z_star:
    if u >= v: continue
    print(u,v, z_star[(u,v)])


gamma = spanning_tree_distribution(z_support, z_star)
print(gamma)

==================== printing z_support =====================
0 2 5
0 11 0
0 13 5
2 9 3
2 10 3
2 13 0
5 11 8
6 11 8
9 13 3
1 10 0
1 12 0
9 10 0
7 12 5
9 12 0
8 9 5
9 16 5
3 4 0
3 6 6
3 8 12
4 5 6
4 7 12
4 8 12
6 14 0
7 8 0
8 16 0
5 15 0
7 16 0
14 15 0
==================== printing z_star (dict) =====================
0 2 0.23529411764705882
0 11 0.9411764705882353
0 13 0.7058823529411764
1 10 0.9411764705882353
1 12 0.9411764705882353
2 9 0.23529411764705882
2 10 0.47058823529411764
2 13 0.9411764705882353
3 4 0.9411764705882353
3 6 0.7058823529411764
3 8 0.23529411764705882
4 5 0.23529411764705882
4 7 0.23529411764705882
4 8 0.47058823529411764
5 11 0.7058823529411764
5 15 0.9411764705882353
6 11 0.23529411764705882
6 14 0.9411764705882353
7 8 0.23529411764705882
7 12 0.47058823529411764
7 16 0.9411764705882353
8 9 0.23529411764705882
8 16 0.7058823529411764
9 10 0.47058823529411764
9 12 0.47058823529411764
9 13 0.23529411764705882
9 16 0.23529411764705882
14 15 0.9411764705882353
{(0,

In [29]:
from math import ceil, exp
from math import log as ln
import math

z_support = nx.Graph(z_support)
lambda_dict = {(u, v): exp(gamma[(u, v)]) for u, v in z_support.edges()}
nx.set_edge_attributes(z_support, lambda_dict, "weight")
# del gamma, lambda_dict

# Sample 2 * ceil( ln(n) ) spanning trees and record the minimum one
minimum_sampled_tree = None
minimum_sampled_tree_weight = math.inf
for _ in range(2 * ceil(ln(G.number_of_nodes()))):
    sampled_tree = random_spanning_tree(z_support, "weight", seed=None)
    sampled_tree_weight = sampled_tree.size(weight)
    if sampled_tree_weight < minimum_sampled_tree_weight:
        minimum_sampled_tree = sampled_tree.copy()
        minimum_sampled_tree_weight = sampled_tree_weight

# Orient the edges in that tree to keep the cost of the tree the same.
t_star = nx.MultiDiGraph()
for u, v, d in minimum_sampled_tree.edges(data=weight):
    if d == G[u][v]['weight']:
        t_star.add_edge(u, v, weight=d)
    else:
        t_star.add_edge(v, u, weight=d)

# Find the node demands needed to neutralize the flow of t_star in G
node_demands = {n: t_star.out_degree(n) - t_star.in_degree(n) for n in t_star}
nx.set_node_attributes(G, node_demands, "demand")

# Find the min_cost_flow
flow_dict = nx.min_cost_flow(G, "demand")

# Build the flow into t_star
for source, values in flow_dict.items():
    for target in values:
        if (source, target) not in t_star.edges and values[target] > 0:
            # IF values[target] > 0 we have to add that many edges
            for _ in range(values[target]):
                t_star.add_edge(source, target)

# Return the shortcut eulerian circuit
circuit = nx.eulerian_circuit(t_star, source=source)





def _shortcutting(circuit):
    """Remove duplicate nodes in the path"""
    nodes = []
    for u, v in circuit:
        if v in nodes:
            continue
        if not nodes:
            nodes.append(u)
        nodes.append(v)
    nodes.append(nodes[0])
    return nodes


    
ans = _shortcutting(circuit)


total_cost = 0
for i in range(len(ans)-1):
    total_cost += G[ans[i]][ans[i+1]]['weight']

print(f"total cost: {total_cost}")
print(ans)


total cost: 50
[16, 7, 8, 4, 3, 14, 12, 1, 10, 9, 13, 2, 0, 11, 5, 6, 15, 16]
